<div class="pull-right"><img src="https://eant.tech/imagenes/logo.png"/></div/>
<h1> <font color=#736AFF> User defined Functions</h1>

<h2> <font color=#2B65EC> Objetive: eliminate outliers </h2>

In [2]:
def eliminar_outliers (dataframe,X, columns,porc=0):
    #dataframe = dataframe that will be use to eliminate outliers
    #X = original dataframe use to obtain quantile
    #porc = porcentaje for which the outliers will be eliminated, if 0 then the mean will be impute
    #columns = columns in the dataframe
    
    for i in range(0,len(columns)):
        if porc == 0:
        #impute the mean
            dataframe[dataframe.loc[:,columns[i]]  > X.loc[:,columns[i]].quantile(0.9)] =\
                dataframe.loc[:,columns[i]].mean()
        else:
        #delete PORC from dataframe
            dataframe =  dataframe[dataframe.loc[:,columns[i]]< X.loc[:,columns[i]].quantile(porc)]
        
    return(dataframe)

<h2> <font color=#2B65EC> Objetive: divide into 2 new dataframes, one with target variable, and the other with features </h2>

In [3]:
def split(dataframes):
    import pandas as pd
    
    obj = pd.DataFrame(columns=['y_train'])
    IND = pd.DataFrame(columns=['X_train','Name'])
    
    for i in range(0,len(dataframes)):
        obj1 = dataframes.iloc[i].Dataframe.diagnosis
        IND1 = dataframes.iloc[i].Dataframe.drop(['diagnosis'], axis=1)
        name1 = dataframes.iloc[i].Name
        
        obj = obj.append({'y_train': obj1 },ignore_index=True)
        IND =IND.append({'X_train':IND1,'Name': name1},ignore_index=True)
        
    
    return obj,IND

<h2> <font color=#2B65EC> Objective: apply models to the dataframes generated </h2>

In [4]:
def models(X_train,y_train,X_test,y_test):
    #X_train = dataframe with 2 variables, one that contains all the dataframes generated (only features) 
    #            with the analysis the other with the mane of that dataframe
    #y_train = dataframe that contains all the dataframes generated (only target) 
    #X_test = features testing
    #y_test = target testing
    
    import pandas as pd
    import numpy as np
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import GridSearchCV
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.linear_model import  RidgeClassifierCV
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.neighbors import KNeighborsClassifier
    
    clf = pd.DataFrame(columns=['Model','Name'])
    predictions = pd.DataFrame(columns=['Model','Dataframe','Acuraccy'])
     
    ##TREE - decision trees learn from data to approximate 
    #a sine curve with a set of if-then-else decision rules
    #DecisionTreeRegressor 
    clf =clf.append({'Model':GridSearchCV(DecisionTreeClassifier(),dict(max_depth=range(1,20)),scoring='accuracy'), \
                 'Name': 'DecisionTreeRegressor'},ignore_index=True)
    
    #Ridge classifier with built-in cross-validation.
    clf = clf.append({'Model': RidgeClassifierCV(alphas=(0.1, 1.0, 10.0)), 'Name': 'RidgeClassifierCV'},\
                     ignore_index=True)
    
    ##GaussianNB
    #Gaussian Naive Bayes algorithm for classification.
    clf = clf.append({'Model': GaussianNB(), 'Name': 'GaussianNB'},ignore_index=True)
    
    ##LinearDiscriminantAnalysis - A classifier with a linear decision boundary, 
    #generated by fitting class conditional densities to the data and using Bayes’ rule.
    clf = clf.append({'Model': LinearDiscriminantAnalysis(), 'Name': 'LinearDiscriminantAnalysis'},ignore_index=True)
    
    ##NEIGHBORS
    #KNeighborsClassifier
    clf = clf.append({'Model': GridSearchCV(KNeighborsClassifier(),dict(n_neighbors=range(1,50)))\
                      , 'Name':'KNeighborsClassifier'},ignore_index=True)
    
 
    for i in range(0,len(clf)):
        for j in range(len(X_train)):
            try:
                model = clf.iloc[i].Model.fit(X_train.iloc[j].X_train, y_train.iloc[j].y_train)
                y_pred = model.predict(X_test[X_train.iloc[j].X_train.columns])
                Acuraccy = np.sqrt(accuracy_score(y_test, y_pred))
                predictions = predictions.append({'Model': clf.iloc[i].Name ,\
                                                  'Dataframe': X_train.iloc[j].Name ,'Acuraccy':Acuraccy},\
                                     ignore_index=True) 
            except: 
                continue
                
    return predictions